#Setup

In [1]:
! pip install pkbar

In [2]:
from google.colab import drive
from google.colab import files
import sys
import time

drive.mount('/content/gdrive/', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Colab Notebooks/MS Thesis/PSGD Paper/'
dpsgd_dir = base_dir + 'DPSGD/'
results_dir = base_dir + 'FNN/results_CURVES/'
logs_dir = base_dir + 'log'
sys.path.append(base_dir)
import preconditioned_stochastic_gradient_descent as psgd 

Mounted at /content/gdrive/


In [3]:
import matplotlib.pyplot as plt
import torch
from torch.autograd import grad
import torch.nn.functional as F
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import plotly.graph_objects as go
# from hessian_torch import Hessian
import numpy as np
import time
import math

import pkbar
from tabulate import tabulate
import scipy.io
from sklearn import metrics
import plotly.express as px
from torchsummary import summary


# Plot Functions

In [4]:
def check_model(data):
    image ,_ = data 
    a = myModel(image.to(device).view(-1,784))
    plt.figure(1)
    plt.imshow(image[0].view(28,28), cmap = 'gray')
    plt.figure(2)
    plt.imshow(a.view(28,28).cpu().detach().numpy(), cmap = 'gray')

def plot_loss_metrics(xaxis,yaxis,title, x_label,y_label):
 
  fig = go.Figure()
  i = 0
  if(xaxis != None):
    for opt in opts:
      fig.add_trace(go.Scatter(x = xaxis[opt], y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1
  else:
    for opt in opts:
      fig.add_trace(go.Scatter(y=yaxis[opt], name = opt, mode='lines', line = dict(color = colors[i])))
      i = i + 1

  fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label, yaxis_type="log")
  fig.show()
  fig.write_html(results_dir + title + ".html")



In [5]:
np.random.seed(0)

# Parameter Settings
BATCH_SIZE = 64
test_BATCH_SIZE = 1000
EPOCHS = 20
GAP = 100

# Data Download

In [6]:
!wget http://www.cs.toronto.edu/~jmartens/digs3pts_1.mat

--2021-04-04 05:52:16--  http://www.cs.toronto.edu/~jmartens/digs3pts_1.mat
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 189600400 (181M)
Saving to: ‘digs3pts_1.mat’

digs3pts_1.mat      100%[===================>] 180.82M  47.5MB/s    in 4.2s    

2021-04-04 05:52:21 (43.1 MB/s) - ‘digs3pts_1.mat’ saved [189600400/189600400]



In [7]:
import scipy 
mat = scipy.io.loadmat('/content/digs3pts_1.mat')
X_train,  X_test, _, _ = mat['bdata'], mat['bdatatest'], mat['tdata'], mat['tdatatest']

X_train = torch.Tensor(X_train) # transform to torch tensor
X_test = torch.Tensor(X_test)

train_dataset = torch.utils.data.TensorDataset(X_train, X_train) 
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4, pin_memory = True)


test_dataset = torch.utils.data.TensorDataset(X_test, X_test) 
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = test_BATCH_SIZE, shuffle = True, num_workers = 4, pin_memory = True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



In [8]:
n_batches = np.ceil(len(train_loader.dataset)/BATCH_SIZE)
n_test_batches = np.ceil(len(test_loader.dataset)/test_BATCH_SIZE)

In [9]:
n_batches, n_test_batches

(313.0, 10.0)

In [10]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")
torch.cuda.device_count()
torch.cuda.get_device_name(0)

Running on the GPU


'Tesla T4'

# Model

In [11]:
"""input image size for the original myModel is 32x32, here is 28x28"""
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
dim = [784, 400, 200, 100, 50, 25, 6, 25, 50, 100, 200, 400, 784]
def initialize_weights():
    W1 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[0]+1,  dim[1])).clone().detach().requires_grad_(True)).to(device)
    W2 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[1]+1,  dim[2])).clone().detach().requires_grad_(True)).to(device)
    W3 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[2]+1,  dim[3])).clone().detach().requires_grad_(True)).to(device)
    W4 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[3]+1,  dim[4])).clone().detach().requires_grad_(True)).to(device)
    W5 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[4]+1,  dim[5])).clone().detach().requires_grad_(True)).to(device)
    W6 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[5]+1,  dim[6])).clone().detach().requires_grad_(True)).to(device)
    W7 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[6]+1,  dim[7])).clone().detach().requires_grad_(True)).to(device)
    W8 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[7]+1,  dim[8])).clone().detach().requires_grad_(True)).to(device)
    W9 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[8]+1,  dim[9])).clone().detach().requires_grad_(True)).to(device)
    W10 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[9]+1,  dim[10])).clone().detach().requires_grad_(True)).to(device)
    W11 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[10]+1,  dim[11])).clone().detach().requires_grad_(True)).to(device)
    W12 = torch.nn.init.xavier_uniform_((0.1*torch.randn(dim[11]+1,  dim[12])).clone().detach().requires_grad_(True)).to(device)
    Ws = [W1, W2, W3, W4, W5, W6, W7, W8, W9, W10, W11, W12]
    return Ws

def myModel(x): 
    W1, W2, W3, W4, W5, W6, W7, W8, W9, W10, W11, W12= Ws
   
    x1 = F.relu(x.mm(W1[:-1]) + W1[-1])
    x2 = F.relu(x1.mm(W2[:-1]) + W2[-1])
    x3 = F.relu(x2.mm(W3[:-1]) + W3[-1])
    x4 = F.relu(x3.mm(W4[:-1]) + W4[-1])
    x5 = F.relu(x4.mm(W5[:-1]) + W5[-1])
    x6 = F.relu(x5.mm(W6[:-1]) + W6[-1])
    x7 = F.relu(x6.mm(W7[:-1]) + W7[-1])
    x8 = F.relu(x7.mm(W8[:-1]) + W8[-1])
    x9 = F.relu(x8.mm(W9[:-1]) + W9[-1])
    x10 = F.relu(x9.mm(W10[:-1]) + W10[-1])
    x11 = F.relu(x10.mm(W11[:-1]) + W11[-1])
    y = (x11.mm(W12[:-1]) + W12[-1])
    
    return y


# Loss Function

In [12]:
def train_loss(data, target):
    y = myModel(data)
    loss = F.mse_loss(y, target)
    return loss

def test_loss():
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device).view(-1, 28*28)
            y = myModel(data)
            loss += F.mse_loss(y, data)
            
    return loss.item()/n_test_batches

def save_start_condition(trainlosslist, testlosslist, timelist):
    trainloss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
      data = data.to(device).view(-1, 28*28)
      loss= train_loss(data, data)
      trainloss += loss
      
    timelist.append(0)

    testloss = test_loss()

    trainlosslist.append(trainloss.item()/n_batches)
    testlosslist.append(testloss)
    print('Epoch: {}; train loss: {}; test loss: {},  time: {}'\
    .format(0, trainlosslist[-1], testlosslist[-1], np.sum(timelist)))

## SGD

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0

    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws)
        trainloss += loss
        
        with torch.no_grad():
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*grads[i]

        kbar.update(n, values=[("loss", loss.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()
    TestLoss.append(testloss)
    
    # step_size = 0.01**(1/9)*step_size
    kbar.add(1, values=[("val_loss", testloss)])

scipy.io.savemat(results_dir + 'sgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.0312501096877808; test loss: 0.0313072144985199,  time: 0
Epoch: 1/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0285 - val_loss: 0.0268
Epoch: 2/20
313/313 [==============================] - 2s 8ms/step - loss: 0.0258 - val_loss: 0.0251
Epoch: 3/20
313/313 [==============================] - 2s 8ms/step - loss: 0.0244 - val_loss: 0.0240
Epoch: 4/20
313/313 [==============================] - 2s 8ms/step - loss: 0.0235 - val_loss: 0.0232
Epoch: 5/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0228 - val_loss: 0.0226
Epoch: 6/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0223 - val_loss: 0.0222
Epoch: 7/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0220 - val_loss: 0.0220
Epoch: 8/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0218 - val_loss: 0.0218
Epoch: 9/20
313/313 [==============================] - 2s 8ms/step - loss: 0.0217 - val_loss: 0.0217
Epoch: 10

## Adam

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
m0 = [torch.zeros(W.shape).to(device) for W in Ws]
v0 = [torch.zeros(W.shape).to(device) for W in Ws]
step_size = 0.001
cnt = 0
n = 0
TrainLoss, TestLoss = [], []

times = []
with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws)#, create_graph=True)
        trainloss += loss

        with torch.no_grad():
            lmbd = min(cnt/(cnt+1), 0.9)
            m0 = [lmbd*old + (1.0-lmbd)*new for (old, new) in zip(m0, grads)]
            lmbd = min(cnt/(cnt+1), 0.999)
            v0 = [lmbd*old + (1.0-lmbd)*new*new for (old, new) in zip(v0, grads)]
            for i in range(len(Ws)):
                Ws[i] -= step_size*(m0[i]/torch.sqrt(v0[i] + 1e-8))
            cnt = cnt + 1
        kbar.update(n, values=[("loss", loss.item())])
        n += 1
        
    t1 = time.time() - t0
    times.append(t1)

    testloss = test_loss()

    TrainLoss.append(trainloss.item()/n_batches)
    
    TestLoss.append(testloss)
    
    # step_size = 0.01**(1/9)*step_size
    kbar.add(1, values=[("val_loss", testloss)])

scipy.io.savemat(results_dir + 'adam.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.0312501096877808; test loss: 0.0313072144985199,  time: 0
Epoch: 1/20
313/313 [==============================] - 3s 9ms/step - loss: 0.0218 - val_loss: 0.0216
Epoch: 2/20
313/313 [==============================] - 3s 9ms/step - loss: 0.0204 - val_loss: 0.0195
Epoch: 3/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0181 - val_loss: 0.0166
Epoch: 4/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0156 - val_loss: 0.0150
Epoch: 5/20
313/313 [==============================] - 3s 9ms/step - loss: 0.0147 - val_loss: 0.0144
Epoch: 6/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0141 - val_loss: 0.0138
Epoch: 7/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0135 - val_loss: 0.0132
Epoch: 8/20
313/313 [==============================] - 3s 8ms/step - loss: 0.0128 - val_loss: 0.0124
Epoch: 9/20
313/313 [==============================] - 3s 9ms/step - loss: 0.0119 - val_loss: 0.0114
Epoch: 10

## Full Kronecker

In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
times = []
with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)

    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        trainloss += loss

        v = [torch.randn(W.shape).to(device) for W in Ws]
        Hv = grad(grads, Ws, v)
          
        with torch.no_grad():
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]  
            pre_grads = [psgd.precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
        kbar.update(n, values=[("loss", loss.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()
    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
    # step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, time: {}'\
    #  .format(epoch, TrainLoss[-1], TestLoss[-1], np.sum(times)))

scipy.io.savemat(results_dir + 'Kron.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss, 'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.0312501096877808; test loss: 0.0313072144985199,  time: 0
Epoch: 1/20
313/313 [==============================] - 11s 36ms/step - loss: 0.0228 - val_loss: 0.0148
Epoch: 2/20
313/313 [==============================] - 11s 35ms/step - loss: 0.0101 - val_loss: 0.0073
Epoch: 3/20
313/313 [==============================] - 11s 36ms/step - loss: 0.0059 - val_loss: 0.0051
Epoch: 4/20
313/313 [==============================] - 11s 35ms/step - loss: 0.0045 - val_loss: 0.0043
Epoch: 5/20
313/313 [==============================] - 11s 36ms/step - loss: 0.0039 - val_loss: 0.0038
Epoch: 6/20
313/313 [==============================] - 11s 36ms/step - loss: 0.0034 - val_loss: 0.0034
Epoch: 7/20
313/313 [==============================] - 11s 36ms/step - loss: 0.0031 - val_loss: 0.0032
Epoch: 8/20
313/313 [==============================] - 11s 36ms/step - loss: 0.0029 - val_loss: 0.0030
Epoch: 9/20
313/313 [==============================] - 11s 35ms/step - loss: 0.0027 - val_loss

# DPSGD Approach 1

In [13]:
_tiny = 1.2e-38 
 # pi = (torch.trace(Ql)*Qr.shape[0])/(torch.trace(Qr)*Ql.shape[0])
    # 

    
def precond_grad_kron(Ql, Qr, Grad):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd**0.5))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd**0.5))*IR)

    return P1.mm(Grad).mm(P2)

def precond_grad_scaw(Ql, qr, Grad):
    P1 = Ql.t().mm(Ql)
    P2 = qr*qr
    pi = (torch.trace(P1)*P2.shape[0])/(torch.sum(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd**0.5))*IL)
    P2 = P2 + torch.sqrt((1/pi)*(eta + lambd**0.5)*IR)
    return P1.mm(Grad*(P2))


def update_lambda(loss1, loss2, M, lambd, omega):
    
    r = abs(loss2 - loss1)/(M)
    if r > 3/4:
      lambd = lambd*omega
    elif r < 1/4:
      lambd = lambd / omega
    return lambd


In [ ]:
torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
TrainLoss, TestLoss = [], []
times = []

with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after
eta = 1e-5

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        
        if n % 1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)

        with torch.no_grad():     
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            pre_grads = [precond_grad_kron(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-10:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.mse_loss(myModel(data), data)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)
        kbar.update(n, values=[("loss", loss.item())])
        n = n+1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
    # step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, time: {}' .format(epoch+1, TrainLoss[-1], TestLoss[-1],np.sum(times)))

scipy.io.savemat(results_dir + 'Kron_damped_2.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.0312501096877808; test loss: 0.0313072144985199,  time: 0
Epoch: 1/20
313/313 [==============================] - 12s 40ms/step - loss: 0.0224 - val_loss: 0.0147
Epoch: 2/20
313/313 [==============================] - 12s 39ms/step - loss: 0.0097 - val_loss: 0.0069
Epoch: 3/20
313/313 [==============================] - 12s 39ms/step - loss: 0.0056 - val_loss: 0.0049
Epoch: 4/20
313/313 [==============================] - 12s 39ms/step - loss: 0.0043 - val_loss: 0.0040
Epoch: 5/20
313/313 [==============================] - 12s 39ms/step - loss: 0.0036 - val_loss: 0.0035
Epoch: 6/20
313/313 [==============================] - 12s 39ms/step - loss: 0.0032 - val_loss: 0.0032
Epoch: 7/20
313/313 [==============================] - 12s 39ms/step - loss: 0.0029 - val_loss: 0.0029
Epoch: 8/20
313/313 [==============================] - 12s 39ms/step - loss: 0.0027 - val_loss: 0.0028
Epoch: 9/20
313/313 [==============================] - 12s 39ms/step - loss: 0.0025 - val_loss

## MIXED

In [16]:
def precond_kron(Ql, Qr, Pl, Pr, beta):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    pi = (torch.trace(P1)*P2.shape[0])/(torch.trace(P2)*P1.shape[0])
    IL = torch.ones(P1.shape[0]).to(device)
    IR = (torch.ones(P2.shape[0])).to(device)
    P1 = P1 + torch.diag(torch.sqrt((pi)*(eta + lambd))*IL)
    P2 = P2 + torch.diag(torch.sqrt((1/pi)*(eta + lambd))*IR)

    Pl = beta*Pl + (1-beta)*P1 
    Pr = beta*Pr + (1-beta)*P2 
    # Pl = 2.33*Pl + P1 
    # Pr = 2.33*Pr + P2 

    return [P1, P2, Pl, Pr]

def precond_kron2(Ql, Qr, Pl, Pr, beta):
    P1 = Ql.t().mm(Ql)
    P2 = Qr.t().mm(Qr)
    Pl = beta*Pl + (1-beta)*P1 
    Pr = beta*Pr + (1-beta)*P2 
    return [P1, P2, Pl, Pr]

def precond_grad_kron2(Pl, Pr, Grad):
    return Pl.mm(Grad).mm(Pr)

torch.manual_seed(1)
Ws = initialize_weights()
Qs = [[torch.eye(W.shape[0]).to(device), torch.eye(W.shape[1]).to(device)] for W in Ws]
Ps = [[0*torch.eye(W.shape[0]).to(device), 0*torch.eye(W.shape[1]).to(device)] for W in Ws]
step_size = 0.1
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []
TrainAcc, TestAcc = [], []
times = []
save_start_condition(TrainLoss, TestLoss, times)

lambd = 1
update_after = 5
omega = (19/20)**update_after
eta = 1e-5

beta = 0.3

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    
    t0 = time.time()
    
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws, create_graph=True)
        
        trainloss += loss
        
        if n % 1 == 0:
          v = [torch.randn(W.shape).to(device) for W in Ws]
          Hv = grad(grads, Ws, v)

        with torch.no_grad():     
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            beta = min(n/(n+1), 0.7)
            Qs = [psgd.update_precond_kron(q[0], q[1], dw, dg) for (q, dw, dg) in zip(Qs, v, Hv)]
            Ps = [precond_kron(q[0], q[1], p[0], p[1], beta) for (q, p) in zip(Qs, Ps)]
            pre_grads = [precond_grad_kron2(p[2], p[3], g) for (p, g) in zip(Ps, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]
            if n % update_after == 0 and lambd > 1e-5:
                M = min([0.5*torch.dot(g.view(-1,), step_size*pg.view(-1,)) for (g, pg) in zip(grads, pre_grads)])
                loss2 = F.mse_loss(myModel(data), data)
                loss1 = loss
                lambd = update_lambda(loss1, loss2, M,  lambd, omega)
        kbar.update(n, values=[("loss", loss.item())])
        n = n+1

    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    
    # step_size = 0.01**(1/9)*step_size
    # print('Epoch: {}; train loss: {}; test loss: {}, time: {}' .format(epoch+1, TrainLoss[-1], TestLoss[-1],np.sum(times)))

# scipy.io.savemat(results_dir + 'mod_psgd.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})
scipy.io.savemat(results_dir + 'new_psgd_1.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.0312501096877808; test loss: 0.0313072144985199,  time: 0
Epoch: 1/20
313/313 [==============================] - 15s 48ms/step - loss: 0.0186 - val_loss: 0.0100
Epoch: 2/20
313/313 [==============================] - 15s 48ms/step - loss: 0.0074 - val_loss: 0.0059
Epoch: 3/20
313/313 [==============================] - 15s 48ms/step - loss: 0.0050 - val_loss: 0.0045
Epoch: 4/20
313/313 [==============================] - 15s 48ms/step - loss: 0.0040 - val_loss: 0.0038
Epoch: 5/20
313/313 [==============================] - 15s 48ms/step - loss: 0.0035 - val_loss: 0.0035
Epoch: 6/20
313/313 [==============================] - 15s 48ms/step - loss: 0.0031 - val_loss: 0.0032
Epoch: 7/20
132/313 [===========>..................] - ETA: 8s - loss: 0.0029

KeyboardInterrupt: ignored

##KFAC

In [ ]:
from kfac import KFAC
import torch.nn as nn
import torch.optim as optim

class myModel_K(nn.Module):
    def __init__(self):
        super(myModel_K, self).__init__()

        self.fc1 = nn.Linear(dim[0], dim[1])
        self.fc2 = nn.Linear(dim[1], dim[2])
        self.fc3 = nn.Linear(dim[2], dim[3])
        self.fc4 = nn.Linear(dim[3], dim[4])
        self.fc5 = nn.Linear(dim[4], dim[5])
        self.fc6 = nn.Linear(dim[5], dim[6])
        self.fc7 = nn.Linear(dim[6], dim[7])
        self.fc8 = nn.Linear(dim[7], dim[8])
        self.fc9 = nn.Linear(dim[8], dim[9])
        self.fc10 = nn.Linear(dim[9], dim[10])
        self.fc11 = nn.Linear(dim[10], dim[11])
        self.fc12 = nn.Linear(dim[11], dim[12])


    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        x = F.relu(self.fc7(x))
        x = F.relu(self.fc8(x))
        x = F.relu(self.fc9(x))
        x = F.relu(self.fc10(x))
        x = F.relu(self.fc11(x))
        x = (self.fc12(x))
        
        return x

def test_loss_K(model):
    model.eval()
    loss = 0
    accuracy = 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.to(device)
            # data = data.view(-1, 28*28*1)
            y = model(data)
            loss += F.mse_loss(y, data)

    return loss.item()/n_test_batches

In [ ]:
torch.manual_seed(1)  
Ws = initialize_weights()
model = myModel_K()
model.to(device)
preconditioner = KFAC(model, 0.001, alpha=0.05)
lr0 = 0.5
optimizer = optim.SGD(model.parameters(), lr=lr0)
TrainLoss, TestLoss = [], []

times = []
with torch.no_grad():
  save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    trainloss = 0.0
    n = 0
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    model.train()
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device).view(-1, 28*28*1)
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.mse_loss(output, data)

        trainloss += loss
        
        loss.backward()
        preconditioner.step()
        optimizer.step()

        kbar.update(n, values=[("loss", loss.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)

    TrainLoss.append(trainloss.item()/n_batches)
    
    # lr0 = 0.01**(1/9)*lr0
    optimizer.param_groups[0]['lr'] = lr0
    testloss = test_loss_K(model)
    kbar.add(1, values=[("val_loss", testloss)])

    TestLoss.append(testloss)
    
scipy.io.savemat(results_dir + 'KFAC.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.0312499908593516; test loss: 0.0313072144985199,  time: 0
Epoch: 1/20
  0/313 [..............................] - ETA: 0s - loss: 0.0000e+00

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:795: UserWarning:

Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.



313/313 [==============================] - 13s 41ms/step - loss: 0.0216 - val_loss: 0.0216
Epoch: 2/20
313/313 [==============================] - 13s 40ms/step - loss: 0.0214 - val_loss: 0.0199
Epoch: 3/20
313/313 [==============================] - 13s 40ms/step - loss: 0.0168 - val_loss: 0.0145
Epoch: 4/20
313/313 [==============================] - 13s 41ms/step - loss: 0.0112 - val_loss: 0.0100
Epoch: 5/20
313/313 [==============================] - 13s 40ms/step - loss: 0.0083 - val_loss: 0.0074
Epoch: 6/20
313/313 [==============================] - 13s 40ms/step - loss: 0.0070 - val_loss: 0.0067
Epoch: 7/20
313/313 [==============================] - 13s 40ms/step - loss: 0.0058 - val_loss: 0.0055
Epoch: 8/20
313/313 [==============================] - 13s 40ms/step - loss: 0.0049 - val_loss: 0.0056
Epoch: 9/20
313/313 [==============================] - 13s 41ms/step - loss: 0.0042 - val_loss: 0.0044
Epoch: 10/20
313/313 [==============================] - 13s 41ms/step - loss: 0.0038 

## Shampoo

In [ ]:
limit = 100
def matrix_power(matrix, power):
    # use CPU for svd for speed up
    matrix = matrix.cpu()
    u, s, v = torch.svd(matrix)
    return (u @ s.pow_(power).diag() @ v.t()).cuda()

def initialize_shampoo(W, eps):
  if W.shape[0] >= limit:
    ql = eps*torch.ones(W.shape[0]).to(device)
  else:
    ql = eps*torch.eye(W.shape[0]).to(device)

  if W.shape[1] >= limit:
    qr = eps*torch.ones(W.shape[1]).to(device)
    
  else:
    qr = eps*torch.eye(W.shape[1]).to(device)
  return ql, qr

def update_shampoo(ql, qr, g):
  if ql.shape[0] >= limit:
    new_ql = ql + torch.diag(g.mm(g.t()))
    inv_ql = torch.pow(new_ql, -1/4)
  else:
    new_ql = ql + torch.diag(g.mm(g.t()))
    inv_ql = matrix_power(new_ql, -1/4)

  if qr.shape[0] >= limit:
    new_qr = qr + torch.diag(g.t().mm(g))
    inv_qr = torch.pow(new_qr, -1/4)
  else:
    new_qr = qr + g.t().mm(g)
    inv_qr = matrix_power(new_qr, -1/4)
  
  return new_ql, new_qr, inv_ql, inv_qr

def shampoo_grads(ql, qr, g):
  if ql.shape[0] >= limit and qr.shape[0] >= limit:
    pre_grads = ql.reshape(-1, 1) * g * qr.reshape(1, -1)
  elif ql.shape[0] >= limit and qr.shape[0] <= limit:
    pre_grads = ql.reshape(-1,1) * (g @ qr)
  elif ql.shape[0] <= limit and qr.shape[0] >= limit:
    pre_grads = (ql @ g) * qr.reshape(1,-1)
  else:
    pre_grads = ql @ g @ qr
  return pre_grads
  

torch.manual_seed(1)
Ws = initialize_weights()
Qs = [initialize_shampoo(W, 0.01) for W in Ws]
step_size = 0.5
grad_norm_clip_thr = 0.1*sum(W.shape[0]*W.shape[1] for W in Ws)**0.5
TrainLoss, TestLoss = [], []

times = []

save_start_condition(TrainLoss, TestLoss, times)

for epoch in range(EPOCHS):
    kbar = pkbar.Kbar(target=n_batches, epoch=epoch, num_epochs=EPOCHS, width=30, always_stateful=False, interval = 1)
    trainloss = 0.0
    n = 0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
      
        data = data.to(device).view(-1,28*28)
        loss  = train_loss(data, data)
        
        grads = grad(loss, Ws)
        
        trainloss += loss
        
        
        with torch.no_grad():
            Qs = [update_shampoo(q[0], q[1], g) for (q, g) in zip(Qs, grads)]
            pre_grads = [shampoo_grads(q[2], q[3], g) for (q, g) in zip(Qs, grads)]
            grad_norm = torch.sqrt(sum([torch.sum(g*g) for g in pre_grads]))
            step_adjust = min(grad_norm_clip_thr/(grad_norm + 1.2e-38), 1.0)
            for i in range(len(Ws)):
                Ws[i] -= step_adjust*step_size*pre_grads[i]

        kbar.update(n, values=[("loss", loss.item())])
        n += 1
    t1 = time.time() - t0
    times.append(t1)
    TrainLoss.append(trainloss.item()/n_batches)
    
    testloss = test_loss()

    TestLoss.append(testloss)
    kbar.add(1, values=[("val_loss", testloss)])
    # step_size = 0.1**(1/9)*step_size

scipy.io.savemat(results_dir + 'shampoo.mat', {'TrainLoss': TrainLoss, 'TestLoss': TestLoss,  'Time':times})

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning:

This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.



Epoch: 0; train loss: 0.0312501096877808; test loss: 0.0313072144985199,  time: 0
Epoch: 1/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0219 - val_loss: 0.0216
Epoch: 2/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0214 - val_loss: 0.0211
Epoch: 3/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0201 - val_loss: 0.0195
Epoch: 4/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0186 - val_loss: 0.0179
Epoch: 5/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0173 - val_loss: 0.0165
Epoch: 6/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0159 - val_loss: 0.0155
Epoch: 7/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0152 - val_loss: 0.0150
Epoch: 8/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0148 - val_loss: 0.0148
Epoch: 9/20
313/313 [==============================] - 4s 14ms/step - loss: 0.0144 - val_loss: 0.0144


# Comparison

In [ ]:
opts = ['sgd','adam','KFAC','shampoo','Kron', 'Kron_damped_2', 'mod_psgd']
total_train_time = {}
opts_data = {}
times = {}
train_times = {}
test_times = {}
train_losses = {}
test_losses = {}
train_accs = {}
test_accs = {}


for opt in opts:
	opts_data[opt] = scipy.io.loadmat(results_dir+opt+'.mat')	

In [ ]:
colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#000000','#33E0FF', '#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']
# colors = ['#0000FF','#00FF00','#FF0000','#33F0FF','#FFA833','#FFF933','#000000','#33E0FF','#FF33E6','#D433FF','#888A0B','#8A0B1E','#B498DF','#1B786D']

In [ ]:
for opt in opts:
  # print(opt)
  data = opts_data[opt]
  times[opt] = data.get('Time')
  train_times[opt] = np.cumsum(times[opt])
  test_times[opt] = np.cumsum(times[opt])
  total_train_time[opt] = np.sum(times[opt])
  train_losses[opt] = data.get('TrainLoss').reshape(EPOCHS+1,)
  test_losses[opt] = data.get('TestLoss').reshape(EPOCHS+1,)


In [ ]:
# plot train_losses vs Iterations
plot_loss_metrics(None,train_losses,'Train Loss vs EPOCHS', 'EPOCHS','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(None,test_losses,'Test Loss vs EPOCHS', 'EPOCHS','Test Loss')
# # plot test_losses vs Iterations
plot_loss_metrics(train_times,train_losses,'Train Loss vs Time', 'Time','Train Loss')
# plot test_losses vs Iterations
plot_loss_metrics(test_times,test_losses,'Test Loss vs Time', 'Time','Test Loss')